In [16]:
import numpy as np
import matplotlib.pyplot as plt
from mgtwr.sel_bw import Sel_BW
from mgtwr.model import Model
import pandas as pd
import multiprocessing as mp
import psutil
import shap
del Sel_BW, Model
from mgtwr.sel_bw import Sel_BW
from mgtwr.model import Model


In [17]:
data = pd.read_csv('data\\STNWR运行数据集.csv')
df = data[['Lon', 'Lat', 'T','X1','X2','X3','X4','X5','X6','X7','X8']].values
coords = df[:,:3]
X0 = df[:,3:]
# coords = data[['Lon', 'Lat', 'T']].values
# X = data[['X1','X2','X3','X4','X5','X6','X7','X8']].values
X1 = np.log(X0 + 1e-20)
X = (X1 - np.mean(X1, axis=0)) / np.std(X1, axis=0)
y0 = data[['Y3']].values
y = np.log(y0 + 1e-20)
print(np.mean(y, axis=0),np.std(y, axis=0))
y = (y - np.mean(y, axis=0)) / np.std(y, axis=0)
sel = Sel_BW(coords, y, X, mode='gtwr', spherical=True, fixed=False)
print('searching...')
with mp.Pool(psutil.cpu_count()) as pool:
    bw, tau = sel.search(bw_min=150,tau_min=0,tau_max=1e6, verbose=True, max_iter=500)
print('bw:', bw, 'tau:', tau)


[10.61557363] [1.76259362]
searching...
Bandwidth:      521.0, tau: 381970.00000, score: 1763.233
Bandwidth:      521.0, tau: 618030.00000, score: 1763.233
Bandwidth:      749.0, tau: 381970.00000, score: 2041.130
Bandwidth:      749.0, tau: 618030.00000, score: 2041.130
Bandwidth:      379.0, tau: 236068.91910, score: 1524.509
Bandwidth:      379.0, tau: 381970.00000, score: 1524.509
Bandwidth:      521.0, tau: 236068.91910, score: 1763.233
Bandwidth:      292.0, tau: 145901.08090, score: 1308.455
Bandwidth:      292.0, tau: 236068.91910, score: 1308.455
Bandwidth:      379.0, tau: 145901.08090, score: 1524.509
Bandwidth:      237.0, tau: 90171.24503, score: 1130.038
Bandwidth:      237.0, tau: 145901.08090, score: 1130.038
Bandwidth:      292.0, tau: 90171.24503, score: 1308.455
Bandwidth:      204.0, tau: 55729.83587, score: 993.422
Bandwidth:      204.0, tau: 90171.24503, score: 993.422
Bandwidth:      237.0, tau: 55729.83587, score: 1130.038
Bandwidth:      183.0, tau: 34442.71046

In [18]:
gtwr = Model(coords, y, X, bw=bw, tau=tau, mode='gtwr', spherical=True, fixed=False)
with mp.Pool(int(psutil.cpu_count() / 2)) as pool:
    gtwr_results = gtwr.fit(pool=pool)

In [19]:
def pre(var):
    var = var.reshape(-1,11)
    predy2 = []
    for i in range(var.shape[0]):
        row = var[i:i+1,:]
        coords2 = row[:,:3]
        Xv1 = row[:,3:]
        Xv2 = np.log(Xv1 + 1e-20)
        Xv2 = (Xv1 - np.mean(X1, axis=0)) / np.std(X1, axis=0)
        predy,_ = gtwr.predict(coords2,Xv2)
        predy = np.exp(predy * np.std(y, axis=0) +np.mean(y, axis=0))
        predy2.append(np.array(predy))
    return np.array(predy2).reshape(-1,)

In [20]:
import numpy as np
from joblib import Parallel, delayed  # 需要安装 joblib: pip install joblib

def pre(var):
    # --- 预计算全局参数（避免循环内重复计算） ---
    X1_mean = np.mean(X1, axis=0)  # X1 是外部定义的训练数据
    X1_std = np.std(X1, axis=0)
    y_mean = np.mean(y, axis=0)    # y 是外部定义的标签数据
    y_std = np.std(y, axis=0)
    
    var = var.reshape(-1, 11)
    
    # --- 定义单行处理的辅助函数 ---
    def process_single_row(row):
        coords2 = row[:, :3]
        Xv1 = row[:, 3:]
        # 特征变换
        Xv2 = np.log(Xv1 + 1e-20)
        Xv2 = (Xv1 - X1_mean) / X1_std  # 使用预计算的均值和标准差
        # 预测
        predy, _ = gtwr.predict(coords2, Xv2)
        # 逆标准化
        predy = np.exp(predy * y_std + y_mean)
        return predy.ravel()
    
    # --- 并行处理所有行（n_jobs=-1 表示使用所有CPU核心）---
    predy2 = Parallel(n_jobs=-1)(delayed(process_single_row)(var[i:i+1, :]) 
                                 for i in range(var.shape[0]))
    
    return np.concatenate(predy2).reshape(-1)


In [21]:
# med = np.median(df, axis=0).reshape((1, df.shape[1]))
df_pandas = pd.DataFrame(df)
df2 = df_pandas.sample(n=112, random_state=42)
explainer = shap.KernelExplainer(pre,df2)


Using 112 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


In [22]:
# for i in range(1120):
#     shap_values2 = []
#     shap_values2.append(explainer(df[i:i+1,]).values)
#     combined_shap = np.vstack(shap_values2)
#     pd.DataFrame(combined_shap).to_csv('C:\\Users\\YihengSu\\Desktop\\GDP\\{i}.csv', index=False, header=['Lon', 'Lat', 'T','X1','X2','X3','X4','X5','X6','X7','X8'])

In [26]:
shap_values2 = explainer(df[0:2,]).values

  0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
print(shap_values2)

[[nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan]]


In [23]:
i=1
shap_values2 = []
shap_values2.append(explainer(df[i:i+2,]).values)
combined_shap = np.vstack(shap_values2)


  0%|          | 0/2 [00:00<?, ?it/s]

d:\program\anaconda\envs\SMGTWR\lib\site-packages\shap\explainers\_kernel.py:633: RuntimeWarning: invalid value encountered in subtract
  eyAdj = self.linkfv(self.ey[:, dim]) - self.link.f(self.fnull[dim])
d:\program\anaconda\envs\SMGTWR\lib\site-packages\shap\explainers\_kernel.py:681: RuntimeWarning: invalid value encountered in multiply
  eyAdj2 = eyAdj - self.maskMatrix[:, nonzero_inds[-1]] * (
d:\program\anaconda\envs\SMGTWR\lib\site-packages\shap\explainers\_kernel.py:681: RuntimeWarning: invalid value encountered in subtract
  eyAdj2 = eyAdj - self.maskMatrix[:, nonzero_inds[-1]] * (


In [24]:
pd.DataFrame(combined_shap).to_csv(f'C:/Users/YihengSu/Desktop/GDP/{i}.csv', index=False, header=False)

In [25]:
# shap_values2 = []
# shap_values2.append(explainer(df[-112:,]).values)
# combined_shap = np.vstack(shap_values2)
# pd.DataFrame(combined_shap).to_csv('C:\\Users\\YihengSu\\Desktop\\CE\\valuesi112.csv', index=False, header=False)